In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from PIL import Image

import random
import skimage
import skimage.feature
import skimage.viewer
import numpy as np

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\skimage\viewer\utils\__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  from .core import *


In [2]:
os.makedirs("Collected_Images/Sentinel", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/Bands", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/Bands/Cropped", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/FRM_pics", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/GAN_pics", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/SAT_pics", exist_ok = True)

In [3]:
# Creating the base url for sentinel hub playground so that 
# the program can create the correct urls to pull images from
base_url = "https://apps.sentinel-hub.com/sentinel-playground/?source=S2&"
lat = "lat=%s&"
lng = "lng=%s&"
zoom = "zoom=%s&"
preset = "preset=%s&"
other_1 = "layers=B01,B02,B03&maxcc=%s&gain=1.0&gamma=1.0&"
other_2 = "time=2019-12-01|2020-06-25&atmFilter=&showDates=false"

full_url = base_url+lat+lng+zoom+preset+other_1+other_2

In [4]:
# Names of the areas that we are going to be pulling from
names = [
         'Nebraska',
         'Illinois',
         'Arizona_Phoenix',
         'Chihuahua_N',
         'Wyoming',
         'Texas',
         'W_Virginia',
         'New_Mexico_Franklin',
         'Colorodo'
        ]

# The different preset bands that are available on the sentinel playground
bands = [
         '1-NATURAL-COLOR',
         '2_COLOR_INFRARED__VEGETATION_',
         '3_FALSE_COLOR__URBAN_',
         '4_AGRICULTURE',
         '5_VEGETATION_INDEX',
         '6_MOISTURE_INDEX',
         '7_GEOLOGY',
         '8_BATHYMETRIC',
         '90_ATMOSPHERIC_PENETRATION',
         '91_SWIR',
         '92_NDWI',
         '93-SWIR-2-11-12'
        ]

In [38]:
lat_off = -0.145
lng_off =  0.173

# The given latitude coordinates for the areas that we are looking at.
lat = [ 
        41.5085 + lat_off,
        43.0688 + lat_off,
        33.7243 + lat_off,   
        29.8406 + lat_off,  
        41.5088 + lat_off,
        31.6533 + lat_off,
        37.1603 + lat_off,
        33.1375 + lat_off,
        37.7185 + lat_off
      ]

# The given longitude coordinates for the areas that we are looking at.
lng = [
        -96.3281 + lng_off,
        -88.2421 + lng_off,
       -112.1484 + lng_off, 
       -105.4687 + lng_off,
       -108.6328 + lng_off,
       -105.8203 + lng_off,
        -81.5624 + lng_off,
       -106.8749 + lng_off,
       -107.2265 + lng_off
      ]

# The desired maximum cloud coverage for the areas that we are looking at.
cc = [
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        5,
        10
]

# The desired zoom level for the sentinel playground images
z = 11

In [47]:
# This cell is to start getting the screenshots from sentinel hub with the different
# bands.

save = "Collected_Images/Sentinel/Bands/"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(2200, 1800)

j = 0
band = bands[j]

for i in range(0, len(names)):
    for band in bands:
        cur_link = full_url % (lat[i], lng[i], z, band, cc[i])
        driver.get(cur_link)
        time.sleep(10)
        driver.refresh()
        time.sleep(10)
        driver.save_screenshot(save + str(names[i]) + "-" + str(band) + ".png")
#         print(cur_link)
driver.close()

[WDM] - Cache is valid for [07/07/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\lawso\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


In [48]:
save = "Collected_Images/Sentinel/"

In [49]:
# Here the collected band images are going to be cropped so that they are most
# aligned with the formation/satelite that were collected in the notebook
# "Desert, Mountain, and Urban Dataset" 


# l = [402, 532, 770, 492, 620, 802, 746, 462, 464]
# r = [642, 512, 274, 552, 424, 242, 298, 582, 580]
# t = [ 60,  60,  60,  60,  60,  60,  60,  60,  60]
# b = [ 28,  28,  28,  28,  28,  28,  28,  28,  28]

names = [
         'Nebraska',
         'Illinois',
         'Arizona_Phoenix',
         'Chihuahua_N',
         'Wyoming',
         'Texas',
         'W_Virginia',
         'New_Mexico_Franklin',
         'Colorodo'
        ]
j = 0
for i in range(0,len(names)):
    img = Image.open("Collected_Images/idk_yet/GAN_pics/" + names[i] + "-gan.png")
    img = img.crop((0,0,img.width/2,img.height))
    img.save(save + "SAT_pics/" + names[i] + "_sat.png")

    wi, hi = img.size

    for j in range(0, len(bands)):
        sentinel_img = Image.open(save + "Bands/"+ names[i]+ "-" + bands[j] + ".png")
        w, h = sentinel_img.size

        offset_w_l = 522
        offset_w_r = 522
        offset_h_t = 60
        offset_h_b = 28

        left = 0 + offset_w_l
        top = 0 + offset_h_t
        right = w - offset_w_r
        bottom = h - offset_h_b

        sentinel_img = sentinel_img.crop((left, top, right, bottom))
        w, h = sentinel_img.size
#     print(sentinel_img.size)

#     jnt = Image.new("RGB", (w + w, h+h))
#     jnt.paste(img, (0,0))
#     jnt.paste(sentinel_img, (0,h))
#     jnt.paste(img,(w,h))
#     jnt.paste(sentinel_img, (w,0))

#     jnt.show()


        sentinel_img.save(save + "Bands/Cropped/" + names[i] + "_" + bands[j] + ".png")

In [50]:
# Next to we will get the formation images from the previous notebook into
# a local folder

for i in range(0,len(names)):
    img = Image.open("Collected_Images/idk_yet/GAN_pics/" + names[i] + "-gan.png")
    img = img.crop((img.width/2, 0, img.width, img.height))
    img.save(save + "FRM_pics/" + names[i] + "_frm.png")

In [51]:
j = 0

for i in range(0, len(names)):
    frm_img = Image.open(save + "FRM_pics/" + names[i] + "_frm.png")
    jnt = Image.new("RGB", (frm_img.width*2, frm_img.height))
    jnt.paste(frm_img, (frm_img.width,0))
    #for j in range(0,len(bands)):
    band_img = Image.open(save + "Bands/Cropped/" + names[i] + "_" + bands[j] + ".png")
    jnt.paste(band_img, (0,0))
    jnt.save(save + "GAN_pics/" + names[i] + "_" + bands[j] + "_gan.png")

In [52]:
for i in range(0, len(names)):
    img = Image.open("Collected_Images/idk_yet/GAN_pics/" + names[i] + "-gan.png")
    sat_img = img.crop((0,0,img.width/2,img.height))
    sat_img.save("Collected_Images/Sentinel/SAT_pics/" + names[i] + "_sat.png")

In [53]:
j = 0
for i in range(0,len(names)):
    sat_img = Image.open("Collected_Images/Sentinel/SAT_pics/" + names[i] + "_sat.png")
    bnd_img = Image.open("Collected_Images/Sentinel/Bands/Cropped/" + names[i] + "_" + bands[j] + ".png")
    jnt = Image.new("RGB", (sat_img.width*2, sat_img.height*2))
    jnt.paste(sat_img, (0,0))
    jnt.paste(sat_img, (sat_img.width, sat_img.height))
    jnt.paste(bnd_img, (0, sat_img.height))
    jnt.paste(bnd_img, (sat_img.width, 0))
    
    jnt.save(save + "Band_sat_pics/" + names[i] + "_sat_bnd.png")
    